In [10]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentExecutor, create_openai_functions_agent


In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
MODEL = "gpt-3.5-turbo"
llm = ChatOpenAI(model=MODEL, temperature=0)

In [19]:
query = """
    Please provide the following details:
    1. What is the capital of Australia?
    2. Who is the current Prime Minister of Australia?
    3. In which city was the current Prime Minister of Australia born?
    4. What is the population of the city where the current Prime Minister of Australia was born?
    5. What is the GDP of Australia?
    6. What are the main industries in the birthplace city of the current Prime Minister of Australia?
    7. Who was the Prime Minister of Australia before the current one and where was he or she born?
    8. What was the GDP of Australia during the term of the previous Prime Minister?
    9. How did the main industries in the birthplace city of the previous Prime Minister contribute to the GDP of Australia during his or her term?
"""



In [20]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults()]

In [21]:
tools = load_tools(["google-serper", "llm-math"], llm=llm)
react_agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [22]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a general agent."),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
agent = AgentExecutor(
    agent=create_openai_functions_agent(llm, tools, prompt), 
    tools=tools, 
    handle_parsing_errors=True, 
    verbose=False,
)

In [23]:
response = react_agent.invoke(query)
print(response['output'])



> Entering new AgentExecutor chain...
This question requires a mix of general knowledge and current information. We will need to use the Google Search API for most of the answers.

Action: google_serper
Action Input: "capital of Australia"
Observation: Australian Capital Territory: Australian Territory. The Australian Capital Territory houses Canberra, Australia’s capital, built between Sydney and Melbourne in the early 20th century. The federal district’s forest, farmland and nature reserves earn Canberra its nickname, the "Bush Capital.” The... Australian Capital Territory Capital: Canberra. Australian Capital Territory Population: 431,215 (Sep 2020). Australian Capital Territory Area: 910 mi². Australian Capital Territory Chief minister: Andrew Barr. Australian Capital Territory Bird: Gang-gang cockatoo. Australian Capital Territory Colour(s): Blue and gold. Australian Capital Territory HDI (2021): 0.980; very high · 1st. The capital city was founded and formally named as Canberra

In [27]:
response = agent.invoke({
    "messages": [
        HumanMessage(content=query),
    ],
})

print(response['output'])

The capital of Australia is Canberra.
